In [180]:
import pandas as pd
df = pd.read_csv('fulldata.csv')
df

/var/folders/tn/ddzd66z517378n9c45psbsb00000gn/T/ipykernel_12748/2711397601.py:2: DtypeWarning: Columns (7,8,12,15,18,20,22,32,50,51,53,54,58,59,60,62,63,64,65,67,68,69,71,72,74,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('fulldata.csv')


,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
0,C00370007,DEMOCRACY FOR AMERICA,2015.0,M10,2.016013e+17,11AI,VNW03E0AAN5,1044354,NaN,IND,INDIVIDUAL,NaN,NaN,"RAMPINI, FEDERICO",Q,NaN,U,FEDERICO,NaN,RAMPINI,NaN,25 CENTRAL PARK W,APT 31K,NEW YORK,NY,100237200,GRUPPO EDITORIALE,JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2015-09-04 00:00:00,25.00,225.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,VNW03E0AAN5E,SA11AI,F3X,4.012920e+18,t,EARMARKED: CONDUIT INFORMATION BELOW,2016.0,SA,NaN,4.012920e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201601...,Contributions From Individuals/Persons Other T...
1,C00499947,FRIENDS OF DAN KILDEE,2016.0,Q2,2.019073e+17,11AI,VTE509R4ZG1,1345334,NaN,IND,INDIVIDUAL,NaN,NaN,"GALLIOT, LORENA",H,NaN,P,LORENA,NaN,GALLIOT,NaN,1061 BERGEN ST,APT 16,BROOKLYN,NY,112163381,THE YEARS PROJECT,JOURNALIST,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-06 00:00:00,0.01,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P2016,NaN,PRIMARY,2016.0,A,ADD,ITEMIZED RECEIPTS,2019-08-02 03:41:11.937,NaN,VTE509R4ZG1E,SA11AI,F3,4.073120e+18,t,* EARMARKED CONTRIBUTION: SEE BELOW,2016.0,SA,NaN,4.080120e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201907...,Contributions From Individuals/Persons Other T...
2,C00608174,ZEPHYR TEACHOUT FOR CONGRESS,2016.0,30G,2.016121e+17,11AI,VR0T7JVN1W9,1133546,NaN,IND,INDIVIDUAL,C00401224,NaN,"MICHEL, AMANDA",H,NaN,P,AMANDA,NaN,MICHEL,NaN,239 W 15TH ST,APT 4FE,NEW YORK,NY,100116452,THE GUARDIAN,JOURNALIST,C00401224,15E,EARMARKED CONTRIBUTION,NaN,NaN,NaN,2016-11-08 00:00:00,10.00,661.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G2016,NaN,GENERAL,2016.0,A,ADD,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,VR0T7JVN1W9E,SA11AI,F3,4.120820e+18,t,* EARMARKED CONTRIBUTION: SEE BELOW,2016.0,SA,NaN,4.010620e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201612...,Contributions From Individuals/Persons Other T...
3,C00575795,HILLARY FOR AMERICA,2016.0,M10,2.017011e+17,17A,C11527071,1137625,NaN,IND,INDIVIDUAL,NaN,NaN,"HOAGLAND, DOUG",P,NaN,P,DOUG,NaN,HOAGLAND,NaN,4696 N 4TH ST,NaN,FRESNO,CA,937261617,RETIRED,JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2016-09-29 00:00:00,8.00,311.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G2016,NaN,GENERAL,2016.0,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3P,4.010920e+18,t,NaN,2016.0,SA,NaN,4.01112

In [181]:
df['contributor_employer'] = df['contributor_employer'].replace(['SELF'], 'SELF-EMPLOYED')
df['contributor_employer'] = df['contributor_employer'].replace(['SELF EMPLOYED'], 'SELF-EMPLOYED')
df['contributor_employer'] = df['contributor_employer'].replace(['HERALD NEWSPAPERS, INC.'], 'HERALD NEWSPAPERS')
df['contributor_employer'] = df['contributor_employer'].replace(['HERALD NEWSPAPERS, INC'], 'HERALD NEWSPAPERS')
df['contributor_employer'] = df['contributor_employer'].replace(['HERALD NEWSPAPERS INC'], 'HERALD NEWSPAPERS')

df['contributor_employer'] = df['contributor_employer'].replace(['FREELANCE'], 'SELF-EMPLOYED')
df['contributor_employer'] = df['contributor_employer'].replace(['FREE LANCE'], 'SELF-EMPLOYED')
df['contributor_employer'] = df['contributor_employer'].replace(['NONE'], 'SELF-EMPLOYED')
df['contributor_employer'] = df['contributor_employer'].replace(['HERALD NEWSPAPERS INC '], 'HERALD NEWSPAPERS')
df['contributor_employer'] = df['contributor_employer'].replace(['SELF- FREELANCE'], 'SELF-EMPLOYED')

## Look for change of total amount of donation by year

In [193]:
df1 = df.loc[df['contributor_employer'] == 'SELF-EMPLOYED']
df2 = df.loc[df['contributor_employer'] != 'SELF-EMPLOYED']

In [194]:
df1 = df1[['report_year','contribution_receipt_amount']]
df1 = df1.groupby(by="report_year")['contribution_receipt_amount'].sum().reset_index()
df1 = df1.rename(columns={"contribution_receipt_amount": "Self_employed"})
df2 = df2[['report_year','contribution_receipt_amount']]
df2 = df2.groupby(by="report_year")['contribution_receipt_amount'].sum().reset_index()
df2 = df2.rename(columns={"contribution_receipt_amount": "Non_self_employed"})

In [207]:
change_by_year_df = df1.merge(df2[['report_year,'Non_self_employed']])
change_by_year_df['report_year'] = pd.to_datetime(change_by_year_df['report_year'], format='%Y') 
change_by_year_df

,report_year,Self_employed,Non_self_employed
0,2015-01-01,192569.38,152754.53
1,2016-01-01,716497.82,602647.37
2,2017-01-01,212592.46,172067.19
3,2018-01-01,523425.05,417807.39
4,2019-01-01,315977.60,256333.41
5,2020-01-01,1028931.24,1050596.92
6,2021-01-01,244793.47,168723.07
7,2022-01-01,490405.28,305523.69


In [209]:
change_by_year_df.to_csv('change_by_year.csv')

## Look for which group got the most amount of donation

In [90]:
df.groupby(by="committee_id")['contribution_receipt_amount'].sum().sort_values(ascending=False).head(20)

committee_id
C00401224    1735924.69
C00000935     320971.42
C00575795     303634.68
C00586537     302810.81
C00010603     260538.32
C00703975     215046.06
C00486845     207900.00
C00694323     160597.96
C00003418     143147.75
C00042366     131754.17
C00027466      76126.51
C00744946      70540.94
C00619411      66666.50
C00577130      51647.06
C00696948      50863.79
C00637512      50500.00
C00696526      46060.88
C00736876      41737.23
C00580100      39605.77
C00697441      36429.16
Name: contribution_receipt_amount, dtype: float64

In [91]:
df.loc[df['committee_id'] == 'C00401224'].head(1)

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
107,C00401224,ACTBLUE,2015.0,YE,2.016020e+17,11AI,SA11AI_28949823,1047732,NaN,IND,INDIVIDUAL,NaN,NaN,"MAIRSON, ALAN",W,NaN,U,ALAN,NaN,MAIRSON,NaN,5624 LAMAR ROAD,NaN,BETHESDA,MD,20816,SELF-EMPLOYED,JOURNALIST,NaN,NaN,NaN,EARMARK,NaN,NaN,2015-07-27 00:00:00,10.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.020320e+18,t,EARMARKED FOR BERNIE 2016 (C00577130),2016.0,SA,NaN,4.021620e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201602...,Contributions From Individuals/Persons Other T...


In [92]:
df.loc[df['committee_id'] == 'C00000935'].head(1)

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
250,C00000935,DCCC,2015.0,YE,2.016041e+17,11AI,VT4C3PXQZ09,1059073,NaN,IND,INDIVIDUAL,NaN,NaN,"SHAW, BETH",Y,NaN,U,BETH,NaN,SHAW,NaN,120 W 45TH ST,FL 39,NEW YORK,NY,100364041,SELF-EMPLOYED,FINANCIAL JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2015-12-30 00:00:00,33400.0,33400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G2015,NaN,GENERAL,2015.0,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.040820e+18,t,OPERATING ACCOUNT CONTRIBUTION,2016.0,SA,NaN,4.041120e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201604...,Contributions From Individuals/Persons Other T...


In [93]:
df.loc[df['committee_id'] == 'C00575795'].head(1)

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
3,C00575795,HILLARY FOR AMERICA,2016.0,M10,2.017011e+17,17A,C11527071,1137625,NaN,IND,INDIVIDUAL,NaN,NaN,"HOAGLAND, DOUG",P,NaN,P,DOUG,NaN,HOAGLAND,NaN,4696 N 4TH ST,NaN,FRESNO,CA,937261617,RETIRED,JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2016-09-29 00:00:00,8.0,311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G2016,NaN,GENERAL,2016.0,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3P,4.010920e+18,t,NaN,2016.0,SA,NaN,4.011120e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201701...,Contributions From Individuals/Persons Other T...


In [94]:
df.loc[df['committee_id'] == 'C00586537'].head(1)

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
310,C00586537,HILLARY VICTORY FUND,2016.0,12G,2.017083e+17,11AI,C12356707,1180853,NaN,IND,INDIVIDUAL,NaN,NaN,"JOSEPH, TARA",N,NaN,J,TARA,NaN,JOSEPH,NaN,"3A , 3 OLD PEAK RD",HONG KONG,NaN,NaN,NaN,THOMSON REUTERS,JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2016-10-06 00:00:00,250.0,250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,PRIMARY,NaN,N,NO CHANGE,ITEMIZED RECEIPTS,2017-09-01 05:36:26.222,NaN,NaN,NaN,F3X,4.083120e+18,t,NaN,2016.0,SA,NaN,4.083120e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201708...,Contributions From Individuals/Persons Other T...


## Where or who the donors work for?

In [95]:
df.groupby(by="contributor_employer")['contribution_receipt_amount'].sum().sort_values(ascending=False).head(40)

contributor_employer
SELF-EMPLOYED                      3699687.08
RETIRED                             337889.13
HERALD NEWSPAPERS                   260170.66
NOT EMPLOYED                        135848.68
DOW JONES & CO.                     102326.00
MIDDLE EAST IMAGES                   85200.00
SUSAN HILLGREN                       43500.00
MLB ADVANCED MEDIA                   40501.58
SOSLAND PUBLISHING                   35536.00
ENID NEWS & EAGLE                    31732.77
WALL STREET JOURNAL                  27492.70
NYS PUBLIC EMPLOYEES FEDERATION      25439.36
PORTER ANDERSON MEDIA                20979.39
CONDE NAST                           19225.24
WSIC                                 16774.20
WOMEN COUNT                          16652.82
THE NEW YORKER                       14805.70
INDEPENDENT                          14278.59
PUBLIC WORKS FINANCING               13773.63
LIFTABLE MEDIA                       13270.00
ESPN                                 13115.69
KOHARA GROUP 

In [96]:
df.groupby(by="contributor_employer")['contribution_receipt_amount'].sum().sort_values(ascending=False).tail(40)

contributor_employer
THESPORTINGSNARF.COM                        1.00
WEST VIRGINIA PUBLIC BROADCASTING           1.00
MICHIGAN NEWS NETWORK                       1.00
PERFECTLY LOUD                              1.00
LATE APEX RACING                            1.00
ASL                                         1.00
SUN SENTINEL                                1.00
GRITPOST LLC                                1.00
ST KITTS AND NEVIS OBSERVER                 1.00
GIZORAMA                                    1.00
FIGHTWING                                   1.00
COURRIER INTERNATIONAL                      1.00
CUTLER PUBLICATIONS                         1.00
NPC DAILY                                   1.00
WQAD                                        1.00
THE CHANNEL CO                              1.00
FANCRED                                     1.00
THE TECHNIQUE                               1.00
THE BADGER PROJECT                          1.00
FOREST PUHA                                 1.00

In [97]:
pd.set_option('display.max_columns', None)
df.loc[df['contributor_employer'] == 'SANLIAN MAGAZINE']

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
30700,C00618371,TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE,2018.0,Q3,2.018102e+17,11AI,SA11AI.1120723,1272508,NaN,IND,INDIVIDUAL,NaN,NaN,"PU, SHI",N,NaN,J,SHI,NaN,PU,NaN,8-2-2-301NO. 9 LITANG ROAD,NaN,BEIJING,CA,10221,SANLIAN MAGAZINE,JOURNALIST,NaN,NaN,NaN,CHARGEBACK,NaN,NaN,2018-07-10 00:00:00,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZED RECEIPTS,2018-10-25 04:25:51,NaN,NaN,NaN,F3X,4.101620e+18,t,NaN,2018.0,SA,NaN,4.102420e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201810...,Contributions From Individuals/Persons Other T...


In [98]:
pd.set_option('display.max_columns', None)
df.loc[df['contributor_employer'] == 'TV MODA']

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
9552,C00575795,HILLARY FOR AMERICA,2016.0,M5,2.016073e+17,18,C4405591,1091718,NaN,IND,INDIVIDUAL,NaN,NaN,"SQUILLO, JO",P,NaN,P,JO,NaN,SQUILLO,NaN,100 LINCOLN RD,NaN,MIAMI BEACH,FL,331392013,TV MODA,JOURNALIST,NaN,15J,MEMO (FILER'S % OF CONTRIBUTION GIVEN TO JOIN,NaN,X,NaN,2016-03-26 00:00:00,-204.35,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P2016,NaN,PRIMARY,2016.0,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,C4370531,SA18,F3P,4.080120e+18,t,* HILLARY VICTORY FUND,2016.0,SA,NaN,4.080320e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201607...,Transfers From Other Authorized Committees
10018,C00575795,HILLARY FOR AMERICA,2016.0,M5,2.016073e+17,17A,C3791369,1091718,NaN,IND,INDIVIDUAL,NaN,NaN,"SQUILLO, JO",P,NaN,P,JO,NaN,SQUILLO,NaN,100 LINCOLN RD,NaN,MIAMI BEACH,FL,331392013,TV MODA,JOURNALIST,NaN,NaN,NaN,NaN,NaN,NaN,2016-04-02 00:00:00,10.00,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P2016,NaN,PRIMARY,2016.0,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3P,4.080120e+18,t,NaN,2016.0,SA,NaN,4.080320e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201607...,Contributions From Individuals/Persons Other T...


## Search for some well-known publications

Dow Jones

In [99]:
df.loc[df['contributor_employer'] == 'DOW JONES & CO.']

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
4778,C00003418,REPUBLICAN NATIONAL COMMITTEE,2015.0,M3,1.597031e+10,11AI,2015M03L11AI00684,999280,NaN,IND,INDIVIDUAL,NaN,MRS.,"KIRKPATRICK, MELANIE M. MRS.",Y,NaN,U,MELANIE,M.,KIRKPATRICK,NaN,210 WEST 90TH STREET,NaN,NEW YORK,NY,100241239,DOW JONES & CO.,JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2015-02-11 00:00:00,33400.0,33400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,PRIMARY,NaN,A,ADD,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.032120e+18,t,NaN,2016.0,SA,NaN,4.032420e+18,http://docquery.fec.gov/cgi-bin/fecimg/?159703...,Contributions From Individuals/Persons Other T...
10140,C00003418,REPUBLICAN NATIONAL COMMITTEE,2016.0,M2,2.016082e+17,17,2016M2L17OR00026,1096504,NaN,IND,INDIVIDUAL,NaN,NaN,"KIRKPATRICK, MELANIE M.",Y,NaN,U,MELANIE,M.,KIRKPATRICK,NaN,210 WEST 90TH STREET,NaN,NEW YORK,NY,10024,DOW JONES & CO.,JOURNALIST,NaN,30,CONVENTION ACCOUNT RECEIPT - INDIVIDUAL,NaN,NaN,NaN,2016-01-27 00:00:00,1600.0,1600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,PRIMARY,NaN,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.082220e+18,t,CONVENTION ACCOUNT,2016.0,SA,NaN,4.082220e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201608...,"Other Federal Receipts (Dividends, Interest, e..."
15367,C00003418,REPUBLICAN NATIONAL COMMITTEE,2016.0,M2,2.016082e+17,11AI,2016M2L11AI02026,1096504,NaN,IND,INDIVIDUAL,NaN,MRS.,"KIRKPATRICK, MELANIE M. MRS.",Y,NaN,U,MELANIE,M.,KIRKPATRICK,NaN,210 WEST 90TH STREET,NaN,NEW YORK,NY,10024,DOW JONES & CO.,JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2016-01-27 00:00:00,33400.0,33400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,PRIMARY,NaN,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.082220e+18,t,NaN,2016.0,SA,NaN,4.082220e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201608...,Contributions From Individuals/Persons Other T...
31362,C00003418,REPUBLICAN NATIONAL COMMITTEE,2017.0,M4,2.017042e+17,11AI,SA11A.71216029,1159644,NaN,IND,INDIVIDUAL,NaN,MRS.,"KIRKPATRICK, MELANIE M. MRS.",Y,NaN,U,MELANIE,M.,KIRKPATRICK,NaN,210 WEST 90TH STREET,NaN,NEW YORK,NY,100241239,DOW JONES & CO.,JOURNALIST,NaN,15,CONTRIBUTION,CONTRIBUTION,NaN,NaN,2017-03-01 00:00:00,33900.0,33900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,PRIMARY,NaN,A,ADD,ITEMIZED RECEIPTS,2017-05-31 12:56:33.511916,NaN,NaN,NaN,F3

In [102]:
df.loc[df['contributor_last_name'] == 'KIRKPATRICK'].loc[df['contributor_first_name'] == 'MELANIE' ]

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
4778,C00003418,REPUBLICAN NATIONAL COMMITTEE,2015.0,M3,1.597031e+10,11AI,2015M03L11AI00684,999280,NaN,IND,INDIVIDUAL,NaN,MRS.,"KIRKPATRICK, MELANIE M. MRS.",Y,NaN,U,MELANIE,M.,KIRKPATRICK,NaN,210 WEST 90TH STREET,NaN,NEW YORK,NY,100241239,DOW JONES & CO.,JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2015-02-11 00:00:00,33400.0,33400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,PRIMARY,NaN,A,ADD,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.032120e+18,t,NaN,2016.0,SA,NaN,4.032420e+18,http://docquery.fec.gov/cgi-bin/fecimg/?159703...,Contributions From Individuals/Persons Other T...
10140,C00003418,REPUBLICAN NATIONAL COMMITTEE,2016.0,M2,2.016082e+17,17,2016M2L17OR00026,1096504,NaN,IND,INDIVIDUAL,NaN,NaN,"KIRKPATRICK, MELANIE M.",Y,NaN,U,MELANIE,M.,KIRKPATRICK,NaN,210 WEST 90TH STREET,NaN,NEW YORK,NY,10024,DOW JONES & CO.,JOURNALIST,NaN,30,CONVENTION ACCOUNT RECEIPT - INDIVIDUAL,NaN,NaN,NaN,2016-01-27 00:00:00,1600.0,1600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,PRIMARY,NaN,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.082220e+18,t,CONVENTION ACCOUNT,2016.0,SA,NaN,4.082220e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201608...,"Other Federal Receipts (Dividends, Interest, e..."
11606,C00458844,MARCO RUBIO FOR PRESIDENT,2015.0,Q2,2.015102e+17,17A,SA17.744685,1029436,NaN,IND,INDIVIDUAL,NaN,MS.,"KIRKPATRICK, MELANIE M. MS.",P,NaN,P,MELANIE,M.,KIRKPATRICK,NaN,351 BRUNING ROAD,NaN,NEW HARTFORD,CT,060572401,DOW JONES & CO,JOURNALIST,NaN,15,CONTRIBUTION,CONTRIBUTION,NaN,NaN,2015-04-28 00:00:00,2700.0,2700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P2016,NaN,PRIMARY,2016.0,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3P,4.101520e+18,t,NaN,2016.0,SA,NaN,4.102120e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201510...,Contributions From Individuals/Persons Other T...
15367,C00003418,REPUBLICAN NATIONAL COMMITTEE,2016.0,M2,2.016082e+17,11AI,2016M2L11AI02026,1096504,NaN,IND,INDIVIDUAL,NaN,MRS.,"KIRKPATRICK, MELANIE M. MRS.",Y,NaN,U,MELANIE,M.,KIRKPATRICK,NaN,210 WEST 90TH STREET,NaN,NEW YORK,NY,10024,DOW JONES & CO.,JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2016-01-27 00:00:00,33400.0,33400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,PRIMARY,NaN,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X

NYT

In [103]:
df.loc[df['contributor_employer'] == 'NEW YORK TIMES']

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
3466,C00401224,ACTBLUE,2016.0,M2,2.016022e+17,11AI,SA11AI_37576931,1051660,NaN,IND,INDIVIDUAL,NaN,NaN,"OCHS, PATRICIA",W,NaN,U,PATRICIA,NaN,OCHS,NaN,6 RUE DU PRIEURE,NaN,MAISONS LAFFITTE,NaN,78600,NEW YORK TIMES,JOURNALIST,NaN,NaN,NaN,EARMARK,NaN,NaN,2016-01-31 00:00:00,12.5,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.022220e+18,t,EARMARKED FOR BERNIE 2016 (C00577130),2016.0,SA,NaN,4.030120e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201602...,Contributions From Individuals/Persons Other T...
4555,C00401224,ACTBLUE,2016.0,M2,2.016022e+17,11AI,SA11AI_37576932,1051660,NaN,IND,INDIVIDUAL,NaN,NaN,"OCHS, PATRICIA",W,NaN,U,PATRICIA,NaN,OCHS,NaN,6 RUE DU PRIEURE,NaN,MAISONS LAFFITTE,NaN,78600,NEW YORK TIMES,JOURNALIST,NaN,NaN,NaN,EARMARK,NaN,NaN,2016-01-31 00:00:00,12.5,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.022220e+18,t,EARMARKED FOR DEMOCRACY FOR AMERICA (C00370007),2016.0,SA,NaN,4.030120e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201602...,Contributions From Individuals/Persons Other T...
6855,C00401224,ACTBLUE,2016.0,M6,2.017021e+17,11AI,SA11AI_48617264,1148219,NaN,IND,INDIVIDUAL,NaN,NaN,"OCHS, PATRICIA",W,NaN,U,PATRICIA,NaN,OCHS,NaN,6 RUE DU PRIEURE,NaN,MAISONS LAFFITTE,ZZ,78600,NEW YORK TIMES,JOURNALIST,NaN,NaN,NaN,EARMARK,NaN,NaN,2016-05-14 00:00:00,27.0,39.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.020820e+18,t,EARMARKED FOR BERNIE 2016 (C00577130),2016.0,SA,NaN,4.032820e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201702...,Contributions From Individuals/Persons Other T...
8522,C00370007,DEMOCRACY FOR AMERICA,2016.0,M3,2.016032e+17,11AI,VNW03E7EKP0,1056666,NaN,IND,INDIVIDUAL,NaN,NaN,"OCHS, PATRICIA",Q,NaN,U,PATRICIA,NaN,OCHS,NaN,6 RUE DU PRIEURE,NaN,MAISONS LAFFITTE,NaN,78600,NEW YORK TIMES,JOURNALIST,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-24 00:00:00,-12.5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,VNW03E7EKP0E,SA11AI,F3X,4.031920e+18,t,CREDIT CARD CHARGEBACK; EARMARKED: CONDUIT INF...,2016.0,SA,NaN,4.032120e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201

WSJ

In [104]:
df.loc[df['contributor_employer'] == 'WALL STREET JOURNAL']

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
13655,C00580688,FRIENDS OF JOE HECK,2016.0,30G,2.016121e+17,11AI,NaN,1135411,NaN,IND,INDIVIDUAL,NaN,NaN,"HORVATH, CECILIE",S,NaN,P,CECILIE,NaN,HORVATH,NaN,3327 P STREET NW,NaN,WASHINGTON,DC,200072702,WALL STREET JOURNAL,JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2016-11-01 00:00:00,2700.0,2700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G2016,NaN,GENERAL,2016.0,A,ADD,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,2.121920e+18,NaN,NaN,F3,1.022820e+18,t,NaN,2016.0,SA,NaN,1.022820e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201612...,Contributions From Individuals/Persons Other T...
14769,C00579458,"JEB 2016, INC.",2015.0,Q3,2.017081e+17,17A,SA17.154093,1178877,NaN,IND,INDIVIDUAL,NaN,NaN,"HORVATH, CECILIE",P,NaN,P,CECILIE,NaN,HORVATH,NaN,3327 P ST NW,NaN,WASHINGTON,DC,200072702,WALL STREET JOURNAL,JOURNALIST,NaN,15,CONTRIBUTION,CONTRIBUTION,NaN,NaN,2015-09-30 00:00:00,2700.0,2700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P2016,NaN,PRIMARY,2016.0,N,NO CHANGE,ITEMIZED RECEIPTS,2017-08-17 05:00:25.264,NaN,NaN,NaN,F3P,4.081420e+18,t,NaN,2016.0,SA,NaN,4.081620e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201708...,Contributions From Individuals/Persons Other T...
29751,C00671537,JULIO GONZALEZ FOR CONGRESS,2018.0,Q1,2.018052e+17,11AI,SA11AI.4199,1233928,NaN,IND,INDIVIDUAL,NaN,NaN,"FAHEY, MARK",H,NaN,P,MARK,NaN,FAHEY,NaN,"3334 CAPITAL MEDICAL BLVD, STE 400",NaN,TALLAHASSEE,FL,32308,WALL STREET JOURNAL,DATA JOURNALIST,NaN,15,CONTRIBUTION,NaN,NaN,NaN,2018-03-29 00:00:00,700.0,700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P2018,NaN,PRIMARY,2018.0,N,NO CHANGE,ITEMIZED RECEIPTS,2018-05-24 02:49:53.68,NaN,NaN,NaN,F3,4.052320e+18,t,NaN,2018.0,SA,NaN,4.052320e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201805...,Contributions From Individuals/Persons Other T...
45967,C00493924,REINVENTING A NEW DIRECTION POLITICAL ACTION C...,2019.0,MY,2.019073e+17,12,SA.766556.15.1902,1344852,NaN,IND,INDIVIDUAL,NaN,NaN,"HORVATH, CECILIE",Q,NaN,D,CECILIE,NaN,HORVATH,NaN,3327 P ST NW,NaN,WASHINGTON,DC,200072702,WALL STREET JOURNAL,JOURNALIST,NaN,15J,MEMO (FILER'S % OF CONTRIBUTION GIVEN TO JOIN,TRANSFER,X,NaN,2019-06-11 00:00:00,5000.0,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P,NaN,PRIMARY,NaN,A,ADD,ITEMIZED RECEIPTS,2019-08-01 04:12:26.251,NaN,SA12.766565,SA12,F3X,4.073020e+18,t,TRANSFER FROM RAND PAUL VICTORY

In [218]:
pd.set_option('display.max_colwidth', None)
df.loc[df['contributor_employer'] == 'WALL STREET JOURNAL'].loc[df['contributor_name'] == 'COLLINS, ELIZA']['receipt_type_full']

192812    EARMARKED FOR REPUBLICAN PARTY OF ARIZONA, LLC (C00008227)
Name: receipt_type_full, dtype: object

In [223]:
df.loc[df['contribution_receipt_amount'] == 50].loc[df['committee_name'] == 'WINRED'].loc[df['report_year'] == 2021.0]

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
151378,C00694323,WINRED,2021.0,MY,2.021122e+17,11AI,A28D4208714B44D2A96F,1552533,NaN,IND,INDIVIDUAL,C00618371,NaN,"RASTEGARRAZI, REZA",V,NaN,U,REZA,NaN,RASTEGARRAZI,NaN,6150 RESEDA BLVD,APT 211,TARZANA,CA,913357354,MEHR-E-IRAN,JOURNALIST,C00618371,15E,EARMARKED CONTRIBUTION,EARMARKED FOR TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE (C00618371),NaN,NaN,2021-03-06 00:00:00,50.0,220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZED RECEIPTS,2022-01-27 11:08:36.007,NaN,NaN,NaN,F3X,4.122120e+18,t,NaN,2022.0,SA,NaN,4.012620e+18,https://docquery.fec.gov/cgi-bin/fecimg/?202112209470723394,Contributions From Individuals/Persons Other Than Political Committees
151442,C00694323,WINRED,2021.0,MY,2.021122e+17,11AI,A1B4ECD9003294FB18CE,1552533,NaN,IND,INDIVIDUAL,NaN,NaN,"LAWLOR, CHRISTOPHER",V,NaN,U,CHRISTOPHER,NaN,LAWLOR,NaN,4 COLLURA LN,NaN,CLIFTON,NJ,070121659,SELF-EMPLOYED,JOURNALIST,NaN,NaN,NaN,EARMARKED FOR REPUBLICAN NATIONAL COMMITTEE (C00003418),NaN,NaN,2021-04-06 00:00:00,50.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZED RECEIPTS,2022-01-27 11:08:57.123,NaN,NaN,NaN,F3X,4.122120e+18,t,NaN,2022.0,SA,NaN,4.012620e+18,https://docquery.fec.gov/cgi-bin/fecimg/?202112209471005993,Contributions From Individuals/Persons Other Than Political Committees
151858,C00694323,WINRED,2021.0,MY,2.021122e+17,11AI,A7DB39F9AEF9248BB90F,1552533,NaN,IND,INDIVIDUAL,NaN,NaN,"LEGGETT, VANESSA",V,NaN,U,VANESSA,NaN,LEGGETT,NaN,820 MARINER,NaN,LAKEWAY,TX,787344351,SELF-EMPLOYED,JOURNALIST,NaN,NaN,NaN,EARMARKED FOR NRCC (C00075820),NaN,NaN,2021-02-04 00:00:00,50.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZED RECEIPTS,2022-01-27 05:17:40.36,NaN,NaN,NaN,F3X,4.122120e+18,t,NaN,2022.0,SA,NaN,4.012620e+18,https://docquery.fec.gov/cgi-bin/fecimg/?202112209470434077,Contributions From Individuals/Persons Other Than Political Committees
151879,C00694323,WINRED,2021.0,MY,2.021122e+17,11AI,AC27B62C358874B07B82,1552533,NaN,IND,INDIVIDUAL,NaN,NaN,"A, JENNIE",V,NaN,U,JENNIE,NaN,A,NaN,1935 WORCESTER DR,NaN,PITTSBURGH,PA,152431539,THE AMERICAN COMPANY,JOURNALIST,NaN,NaN,NaN,EARMARKED FOR SAVE AMERICA JOINT FUNDRAISING COMMITTEE (C00770941),NaN,NaN,2021-04-11 00:00:00,50.0,155.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZ

The New Yorker

In [105]:
df.loc[df['contributor_employer'] == 'THE NEW YORKER']

,committee_id,committee_name,report_year,report_type,image_number,line_number,transaction_id,file_number,committee_name.1,entity_type,entity_type_desc,unused_contbr_id,contributor_prefix,contributor_name,recipient_committee_type,recipient_committee_org_type,recipient_committee_designation,contributor_first_name,contributor_middle_name,contributor_last_name,contributor_suffix,contributor_street_1,contributor_street_2,contributor_city,contributor_state,contributor_zip,contributor_employer,contributor_occupation,contributor_id,receipt_type,receipt_type_desc,receipt_type_full,memo_code,memo_code_full,contribution_receipt_date,contribution_receipt_amount,contributor_aggregate_ytd,candidate_id,candidate_name,candidate_first_name,candidate_last_name,candidate_middle_name,candidate_prefix,candidate_suffix,candidate_office,candidate_office_full,candidate_office_state,candidate_office_state_full,candidate_office_district,conduit_committee_id,conduit_committee_name,conduit_committee_street1,conduit_committee_street2,conduit_committee_city,conduit_committee_state,conduit_committee_zip,donor_committee_name,national_committee_nonfederal_account,election_type,election_type_full,fec_election_type_desc,fec_election_year,amendment_indicator,amendment_indicator_desc,schedule_type_full,load_date,original_sub_id,back_reference_transaction_id,back_reference_schedule_name,filing_form,link_id,is_individual,memo_text,two_year_transaction_period,schedule_type,increased_limit,sub_id,pdf_url,line_number_label
820,C00401224,ACTBLUE,2015.0,YE,2.016020e+17,11AI,SA11AI_32136458,1047732,NaN,IND,INDIVIDUAL,NaN,NaN,"STILLMAN, SARAH",W,NaN,U,SARAH,NaN,STILLMAN,NaN,2007 PLYMOUTH ST NW,NaN,WASHINGTON,DC,20012,THE NEW YORKER,JOURNALIST,NaN,NaN,NaN,EARMARK,NaN,NaN,2015-10-21 00:00:00,50.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,ADD,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.020320e+18,t,EARMARKED FOR SITTENFELD FOR SENATE (C00571737),2016.0,SA,NaN,4.021620e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201602...,Contributions From Individuals/Persons Other T...
10501,C00401224,ACTBLUE,2015.0,M2,2.015091e+17,11AI,SA11AI_25039429,1024608,NaN,IND,INDIVIDUAL,NaN,NaN,"STILLMAN, SARAH",W,NaN,U,SARAH,NaN,STILLMAN,NaN,2007 PLYMOUTH ST NW,NaN,WASHINGTON,DC,20012,THE NEW YORKER,JOURNALIST,NaN,NaN,NaN,EARMARK,NaN,NaN,2015-01-22 00:00:00,50.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NO CHANGE,ITEMIZED RECEIPTS,2017-05-31 13:27:14.31838,NaN,NaN,NaN,F3X,4.091420e+18,t,EARMARKED FOR SITTENFELD FOR SENATE (C00571737),2016.0,SA,NaN,4.091620e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201509...,Contributions From Individuals/Persons Other T...
22160,C00606939,ROSEN FOR NEVADA,2018.0,12G,2.019102e+17,11AI,VR0S7QJV5C1,1358355,NaN,IND,INDIVIDUAL,C00401224,NaN,"DENBY, DAVID",S,NaN,P,DAVID,NaN,DENBY,NaN,345 W 88TH ST,NaN,NEW YORK,NY,100242205,THE NEW YORKER,JOURNALIST,C00401224,15E,EARMARKED CONTRIBUTION,NaN,NaN,NaN,2018-10-09 00:00:00,200.0,1300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G2018,NaN,GENERAL,2018.0,N,NO CHANGE,ITEMIZED RECEIPTS,2019-10-24 03:43:18.413,NaN,VR0S7QJV5C1E,SA11AI,F3,4.101520e+18,t,* EARMARKED CONTRIBUTION: SEE BELOW,2018.0,SA,NaN,4.102320e+18,http://docquery.fec.gov/cgi-bin/fecimg/?201910...,Contributions From Individuals/Persons Other T...
22164,C00606939,ROSEN FOR NEVADA,2018.0,12G,2.019102e+17,11AI,VR0S7QQ2CX3,1358355,NaN,IND,INDIVIDUAL,C00401224,NaN,"DENBY, DAVID",S,NaN,P,DAVID,NaN,DENBY,NaN,345 W 88TH ST,NaN,NEW YORK,NY,100242205,THE NEW YORKER,JOURNALIST,C00401224,15E,EARMARKED CONTRIBUTION,NaN,NaN,NaN,2018-10-12 00:00:00,200.0,1300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G2018,NaN,GENERAL,2018.0,N,NO CHANGE,ITEMIZED RECEIPTS,2019-10-24 03:43:44.963,NaN,VR0S7QQ2CX3E,SA11AI,F3,4.101520e+18,t,* EARMARKED CONTRIBUTION: SEE B

In [106]:
df.loc[df['contributor_employer'] == 'THE NEW YORKER'].groupby(by="report_year")['contribution_receipt_amount'].sum()

report_year
2015.0     100.0
2017.0    2527.0
2018.0    2100.0
2019.0    1550.0
2020.0    8448.7
2021.0      44.0
2022.0      36.0
Name: contribution_receipt_amount, dtype: float64

## What about self employed journalists?

In [107]:
df.loc[df['contributor_employer'] == 'SELF-EMPLOYED'].groupby(by="report_year")['contribution_receipt_amount'].sum()

report_year
2015.0     191734.88
2016.0     713300.33
2017.0     211379.96
2018.0     520985.02
2019.0     315742.60
2020.0    1015040.54
2021.0     243480.97
2022.0     488022.78
Name: contribution_receipt_amount, dtype: float64

In [211]:
df.loc[df['committee_name'] == 'ACTBLUE'].groupby(by="report_year")['contribution_receipt_amount'].sum()

report_year
2015.0     49080.63
2016.0    175614.04
2017.0     92398.16
2018.0    184730.21
2019.0    199246.08
2020.0    712345.70
2021.0    113558.52
2022.0    208951.35
Name: contribution_receipt_amount, dtype: float64